In [1]:
import pandas as pd
from pandas import read_csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import warnings
warnings.filterwarnings("ignore")

os.chdir('/Users/xiaodanxu/Documents/SynthFirm.nosync')

In [2]:
VIUS_fuel_lookup = {'Gasoline': 'Gasoline', 
                    'Diesel': 'Diesel',
                    'Compressed natural gas': 'CNG',
                    'Propane': 'Other',
                    'Combination': 'Other',
                    'Liquified natural gas': 'Other',
                    'Alcohol fuels': 'Other',
                    'Electricity': 'Electricity'}

# load vius data
vius_data_path = 'RawData/US_VIUS_2021/vius_2021_com_crosswalk_20240624.csv'
vius_fleet = read_csv(vius_data_path)
print('vius sample size')
print(len(vius_fleet))

vius sample size
38584


In [3]:
vius_fleet.loc[:, 'WGT_VMT'] = \
vius_fleet.loc[:, 'TABWEIGHT'] * vius_fleet.loc[:, 'MILESANNL']

vius_fleet.loc[vius_fleet['GVWR_CLASS'].isin(['1', '2A']),'VEH_CLASS_SynthFirm'] = 'Light-duty Class12A'
vius_fleet.loc[vius_fleet['GVWR_CLASS'].isin(['2B', '3']),'VEH_CLASS_SynthFirm'] = 'Light-duty Class2B3'
vius_fleet_with_fuel = \
vius_fleet.loc[vius_fleet['FUELTYPE'] != 'Not reported']
print(len(vius_fleet_with_fuel))
vius_fleet_with_fuel.loc[:, 'fuelTypeName'] = \
vius_fleet_with_fuel.loc[:, 'FUELTYPE'].map(VIUS_fuel_lookup)

veh_classes = vius_fleet.VEH_CLASS_SynthFirm.unique()

vius_VMT_by_state = pd.pivot_table(vius_fleet,
                                     index = 'REGSTATE',
                                     columns = 'VEH_CLASS_SynthFirm',
                                     values = 'WGT_VMT',
                                     aggfunc = 'sum')

# Calculate the sum of each column
sum_row = vius_VMT_by_state.sum()

# Append the sum row to the DataFrame
vius_VMT_by_state.loc['National'] = sum_row
vius_VMT_by_state['Total'] = vius_VMT_by_state.sum(axis=1)

vius_VMT_by_state = vius_VMT_by_state.reset_index()
for veh in veh_classes:
    attr_name = 'Frac_' + veh
    vius_VMT_by_state.loc[:, attr_name] = \
    vius_VMT_by_state.loc[:, veh]/ vius_VMT_by_state.loc[:, 'Total']

vius_VMT_by_state.to_csv('RawData/US_VIUS_2021/VIUS_VMT_by_state.csv',
                          index = False)
vius_VMT_by_state.head(5)

37891


VEH_CLASS_SynthFirm,REGSTATE,HDT tractor,HDT vocational,Light-duty Class12A,Light-duty Class2B3,MDT vocational,Total,Frac_HDT tractor,Frac_MDT vocational,Frac_HDT vocational,Frac_Light-duty Class2B3,Frac_Light-duty Class12A
0,Alabama,1.692872e+09,2.554881e+08,5.734538e+09,3.837202e+08,3.615818e+08,8.428200e+09,0.200858,0.042901,0.030313,0.045528,0.680399
1,Alaska,9.325114e+07,3.643108e+07,3.375124e+08,1.860546e+08,4.494443e+07,6.981936e+08,0.133561,0.064372,0.052179,0.266480,0.483408
2,Arizona,1.888836e+09,3.300395e+08,3.818120e+09,7.325185e+08,2.303405e+08,6.999855e+09,0.269839,0.032906,0.047149,0.104648,0.545457
3,Arkansas,1.325041e+09,1.483374e+08,3.545758e+09,6.096115e+08,1.985830e+08,5.827331e+09,0.227384,0.034078,0.025455,0.104612,0.608470
4,California,1.304404e+10,1.596985e+09,4.868745e+10,8.056272e+09,3.321323e+09,7.470607e+10,0.174605,0.044459,0.021377,0.107840,0.651720


In [11]:
# vehicle count

vius_count_by_state = pd.pivot_table(vius_fleet,
                                     index = 'REGSTATE',
                                     columns = 'VEH_CLASS_SynthFirm',
                                     values = 'TABWEIGHT',
                                     aggfunc = 'sum')

# Calculate the sum of each column
sum_row = vius_count_by_state.sum()

# Append the sum row to the DataFrame
vius_count_by_state.loc['National'] = sum_row
vius_count_by_state['Total'] = vius_count_by_state.sum(axis=1)

vius_count_by_state = vius_count_by_state.reset_index()
for veh in veh_classes:
    attr_name = 'Frac_' + veh
    vius_count_by_state.loc[:, attr_name] = \
    vius_count_by_state.loc[:, veh]/ vius_count_by_state.loc[:, 'Total']

vius_count_by_state.to_csv('RawData/US_VIUS_2021/VIUS_count_by_state_synthfirm_types.csv',
                          index = False)
vius_count_by_state.head(5)

VEH_CLASS_SynthFirm,REGSTATE,HDT tractor,HDT vocational,Light-duty Class12A,Light-duty Class2B3,MDT vocational,Total,Frac_HDT tractor,Frac_MDT vocational,Frac_HDT vocational,Frac_Light-duty Class2B3,Frac_Light-duty Class12A
0,Alabama,28876.8,16192.0,319952.3,48068.3,21882.2,434971.6,0.066388,0.050307,0.037225,0.110509,0.735571
1,Alaska,3942.4,4527.5,47254.3,20122.1,4984.0,80830.3,0.048774,0.061660,0.056012,0.248943,0.584611
2,Arizona,31774.8,18633.1,335601.5,106866.2,17144.2,510019.8,0.062301,0.033615,0.036534,0.209533,0.658017
3,Arkansas,26762.2,9351.0,188802.9,53083.3,15907.8,293907.2,0.091057,0.054125,0.031816,0.180612,0.642390
4,California,236682.0,83681.5,3317871.6,419734.7,207456.2,4265426.0,0.055488,0.048637,0.019619,0.098404,0.777852


In [23]:
# generate long-haul fraction among MHD
# print(vius_fleet)
mhd_type = ['HDT tractor', 'MDT vocational', 'HDT vocational']
vius_fleet_mhd = \
vius_fleet.loc[vius_fleet['VEH_CLASS_SynthFirm'].isin(mhd_type)]
# print(vius_fleet_mhd.VEH_CLASS_SynthFirm.unique())
# print(vius_fleet_mhd.columns)
hauling_mode = vius_fleet_mhd.SHORT_HAUL.unique()
long_haul_fraction = pd.pivot_table(vius_fleet_mhd, 
                                    index = ['REGSTATE', 'VEH_CLASS_SynthFirm'],
                                    values = 'TABWEIGHT', columns = 'SHORT_HAUL',
                                    aggfunc = 'sum')
long_haul_fraction.loc[:, 'total'] = long_haul_fraction.sum(axis = 1)
long_haul_fraction.loc[:, hauling_mode] =\
long_haul_fraction.loc[:, hauling_mode].divide(long_haul_fraction.loc[:, 'total'], axis = 0)
long_haul_fraction.columns = ['long haul', 'short haul', 'total']
long_haul_fraction = long_haul_fraction.reset_index()
long_haul_fraction.fillna(0, inplace = True)

vius_fleet_mhd_size = vius_fleet_mhd.groupby(['REGSTATE', 'VEH_CLASS_SynthFirm']).size()
vius_fleet_mhd_size = vius_fleet_mhd_size.reset_index()
vius_fleet_mhd_size.columns = ['REGSTATE', 'VEH_CLASS_SynthFirm', 'sample size']
long_haul_fraction = pd.merge(long_haul_fraction, vius_fleet_mhd_size,
                              on = ['REGSTATE', 'VEH_CLASS_SynthFirm'], 
                              how = 'left')
sample_size_cut_off = 30
long_haul_fraction = \
long_haul_fraction.loc[long_haul_fraction['sample size'] >= sample_size_cut_off]
long_haul_fraction.to_csv('RawData/US_VIUS_2021/VIUS_count_by_hauling.csv',
                          index = False)
long_haul_fraction.head(5)

,REGSTATE,VEH_CLASS_SynthFirm,long haul,short haul,total,sample size
0,Alabama,HDT tractor,0.572865,0.427135,28876.8,308
1,Alabama,HDT vocational,0.069114,0.930886,16192.0,159
2,Alabama,MDT vocational,0.130837,0.869163,21882.2,137
3,Alaska,HDT tractor,0.293019,0.706981,3942.4,197
4,Alaska,HDT vocational,0.081149,0.918851,4527.5,221


In [10]:
vius_fleet['VEH_CLASS_SynthFirm'].unique()

array(['HDT tractor', 'MDT vocational', 'HDT vocational',
       'Light-duty Class2B3', 'Light-duty Class12A'], dtype=object)

In [12]:
vius_fleet.to_csv('RawData/US_VIUS_2021/vius_2021_com_crosswalk_20250211.csv',
                          index = False)
vius_fleet.head(5)

,ID,TABWEIGHT,REGSTATE,ACQUIREYEAR,ACQUISITION,AVGWEIGHT,BRAKES,BTYPE,BUSRELATED,CAB,...,AGE_BIN,weighted_VMT,VEH_CLASS_4,SHORT_HAUL,RO_GT100,VEH_CLASS_SynthF,VEH_CLASS_SynthFirm,SUT_2B_def,VEH_CLASS_MOVES,WGT_VMT
0,1,38.5,Montana,2011,Used,"100,001 to 130,000 pounds",Air,Not applicable (see 'Applicable Vehicles'),Some commercial activity,Not applicable (see 'Applicable Vehicles'),...,age>20,247324.0,Class 7-8,0,70.0,HDT,HDT tractor,NaN,Combination long-haul truck,247324.0
1,2,197.4,North Carolina,Not reported,Used,"60,001 to 80,000 pounds",Air,Not applicable (see 'Applicable Vehicles'),Some commercial activity,Not applicable (see 'Applicable Vehicles'),...,10<age<=15,9381829.8,Class 7-8,0,60.0,HDT,HDT tractor,NaN,Combination long-haul truck,9381829.8
2,8,182.3,New Jersey,2009,Used,"50,001 to 60,000 pounds",Air,Not applicable (see 'Applicable Vehicles'),Some commercial activity,Not applicable (see 'Applicable Vehicles'),...,age>20,6301017.2,Class 7-8,1,0.0,HDT,HDT tractor,NaN,Combination short-haul truck,6301017.2
3,9,85.9,Louisiana,Not reported,Used,"60,001 to 80,000 pounds",Air,Not applicable (see 'Applicable Vehicles'),Some commercial activity,Not applicable (see 'Applicable Vehicles'),...,15<age<=20,3176839.7,Class 7-8,1,NaN,HDT,HDT tractor,NaN,Combination short-haul truck,3176839.7
4,10,240.2,Ohio,2019,New,"60,001 to 80,000 pounds",Air,Not applicable (see 'Applicable Vehicles'),Some commercial activity,Not applicable (see 'Applicable Vehicles'),...,age<=1,17753662.4,Class 7-8,0,85.0,HDT,HDT tractor,NaN,Combination long-haul truck,17753662.4


In [13]:
vius_fleet_diesel = \
vius_fleet_with_fuel.loc[vius_fleet_with_fuel['fuelTypeName'] == 'Diesel']

vius_fleet_diesel.loc[:, 'old_truck'] = 0
vius_fleet_diesel.loc[vius_fleet_diesel['MODELYEAR'] <= 2010, \
'old_truck'] = 1

agg_var = ['REGSTATE', 'VEH_CLASS_MOVES', 'old_truck']
old_truck_fraction = \
vius_fleet_diesel.groupby(agg_var)[['TABWEIGHT', 'WGT_VMT']].sum()
old_truck_fraction = old_truck_fraction.reset_index()

old_truck_size = \
vius_fleet_diesel.groupby(['REGSTATE', 'VEH_CLASS_MOVES'])[['TABWEIGHT']].count()
old_truck_size = old_truck_size.reset_index()
old_truck_size.rename(columns = {'TABWEIGHT': 'Sample size'}, inplace = True)

old_truck_fraction = pd.merge(old_truck_fraction,
                              old_truck_size, 
                              on = ['REGSTATE', 'VEH_CLASS_MOVES'], how = 'left')

old_truck_fraction.loc[:, 'Count_fraction'] = \
old_truck_fraction.loc[:, 'TABWEIGHT'] / \
old_truck_fraction.groupby(['REGSTATE',	'VEH_CLASS_MOVES'])['TABWEIGHT'].transform('sum')

old_truck_fraction.loc[:, 'VMT_fraction'] = \
old_truck_fraction.loc[:, 'WGT_VMT'] / \
old_truck_fraction.groupby(['REGSTATE',	'VEH_CLASS_MOVES'])['WGT_VMT'].transform('sum')
old_truck_fraction.head(5)

,REGSTATE,VEH_CLASS_MOVES,old_truck,TABWEIGHT,WGT_VMT,Sample size,Count_fraction,VMT_fraction
0,Alabama,Combination long-haul truck,0,11251.3,994798430.5,174,0.689181,0.746484
1,Alabama,Combination long-haul truck,1,5074.3,337846198.9,174,0.310819,0.253516
2,Alabama,Combination short-haul truck,0,5965.8,214936292.7,130,0.490266,0.617106
3,Alabama,Combination short-haul truck,1,6202.7,133360662.1,130,0.509734,0.382894
4,Alabama,Light commercial truck,0,8759.8,93314410.0,4,0.436802,0.201487


In [14]:
# national total
agg_var_us = ['VEH_CLASS_MOVES', 'old_truck']
old_truck_fraction_us = \
vius_fleet_diesel.groupby(agg_var_us)[['TABWEIGHT', 'WGT_VMT']].sum()
old_truck_fraction_us = old_truck_fraction_us.reset_index()

old_truck_size_us = \
vius_fleet_diesel.groupby('VEH_CLASS_MOVES')[['TABWEIGHT']].count()
old_truck_size_us = old_truck_size_us.reset_index()
old_truck_size_us.rename(columns = {'TABWEIGHT': 'Sample size'}, inplace = True)

old_truck_fraction_us = pd.merge(old_truck_fraction_us,
                              old_truck_size_us, 
                              on = 'VEH_CLASS_MOVES', how = 'left')

old_truck_fraction_us.loc[:, 'Count_fraction'] = \
old_truck_fraction_us.loc[:, 'TABWEIGHT'] / \
old_truck_fraction_us.groupby(['VEH_CLASS_MOVES'])['TABWEIGHT'].transform('sum')

old_truck_fraction_us.loc[:, 'VMT_fraction'] = \
old_truck_fraction_us.loc[:, 'WGT_VMT'] / \
old_truck_fraction_us.groupby(['VEH_CLASS_MOVES'])['WGT_VMT'].transform('sum')
old_truck_fraction_us.loc[:, 'REGSTATE'] = 'US total'
old_truck_fraction_us.head(5)

,VEH_CLASS_MOVES,old_truck,TABWEIGHT,WGT_VMT,Sample size,Count_fraction,VMT_fraction,REGSTATE
0,Combination long-haul truck,0,871354.0,8.235199e+10,6272,0.799826,0.840137,US total
1,Combination long-haul truck,1,218076.1,1.567013e+10,6272,0.200174,0.159863,US total
2,Combination short-haul truck,0,507139.9,1.756328e+10,6620,0.512409,0.657347,US total
3,Combination short-haul truck,1,482577.4,9.155163e+09,6620,0.487591,0.342653,US total
4,Light commercial truck,0,749131.2,1.304498e+10,213,0.581042,0.759593,US total


In [15]:
old_truck_fraction_out = pd.concat([old_truck_fraction, old_truck_fraction_us])
old_truck_fraction_out = \
old_truck_fraction_out.loc[old_truck_fraction_out['old_truck'] == 1]
old_truck_fraction_out.head(5)

,REGSTATE,VEH_CLASS_MOVES,old_truck,TABWEIGHT,WGT_VMT,Sample size,Count_fraction,VMT_fraction
1,Alabama,Combination long-haul truck,1,5074.3,337846198.9,174,0.310819,0.253516
3,Alabama,Combination short-haul truck,1,6202.7,133360662.1,130,0.509734,0.382894
5,Alabama,Light commercial truck,1,11294.6,369814073.6,4,0.563198,0.798513
7,Alabama,Single-unit long-haul truck,1,1013.0,14619424.7,27,0.282338,0.101831
9,Alabama,Single-unit short-haul truck,1,18961.4,149952028.6,269,0.543958,0.332294


In [16]:
old_truck_fraction_wide = \
pd.pivot_table(old_truck_fraction_out, 
               index = ['REGSTATE'],
              columns = 'VEH_CLASS_MOVES',
              values = ['Count_fraction','VMT_fraction', 'Sample size'] )
old_truck_fraction_wide = old_truck_fraction_wide.reset_index()

old_truck_fraction_wide.to_csv('RawData/US_VIUS_2021/fraction_of_old_diesel_trucks.csv')